# Data Collection

In [1]:
# Step 1: Import necessary libraries
import nltk
import pandas as pd

# Step 2: Download the Gutenberg corpus from NLTK
nltk.download('gutenberg')

# Step 3: Import the Gutenberg corpus
from nltk.corpus import gutenberg

# Step 4: Load the 'Hamlet' text
data = gutenberg.raw('shakespeare-hamlet.txt')  # This loads the raw text of Hamlet

# Step 5: Save the loaded text into a local file
with open('hamlet.txt', 'w') as file:
    file.write(data)  # Write the full text into 'hamlet.txt' file


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


Now that you have trained and saved your model and tokenizer, you can download them to your local machine for deployment. The following code will create a zip file containing the necessary files.

You can now download the `next_word_prediction_model.zip` file from the Colab file explorer (the folder icon on the left sidebar).

# Data Preprocessing

In [2]:
# Step 1: Import necessary libraries
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Step 2: Load the dataset
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()  # Read the text file and convert all text to lowercase

# Step 3: Initialize the Tokenizer
tokenizer = Tokenizer()  # This will convert words into unique integer indices

# Step 4: Fit the tokenizer on the text
tokenizer.fit_on_texts([text])  # Learn the vocabulary from the text

# Step 5: Get total number of unique words
total_words = len(tokenizer.word_index) + 1  # +1 because index starts from 1
print(f'Total unique words: {total_words}')


Total unique words: 4818


In [3]:
total_words
# how tokenize in entair data set
tokenizer.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'it': 9,
 'in': 10,
 'that': 11,
 'ham': 12,
 'is': 13,
 'not': 14,
 'his': 15,
 'this': 16,
 'with': 17,
 'your': 18,
 'but': 19,
 'for': 20,
 'me': 21,
 'lord': 22,
 'as': 23,
 'what': 24,
 'he': 25,
 'be': 26,
 'so': 27,
 'him': 28,
 'haue': 29,
 'king': 30,
 'will': 31,
 'no': 32,
 'our': 33,
 'we': 34,
 'on': 35,
 'are': 36,
 'if': 37,
 'all': 38,
 'then': 39,
 'shall': 40,
 'by': 41,
 'thou': 42,
 'come': 43,
 'or': 44,
 'hamlet': 45,
 'good': 46,
 'do': 47,
 'hor': 48,
 'her': 49,
 'let': 50,
 'now': 51,
 'thy': 52,
 'how': 53,
 'more': 54,
 'they': 55,
 'from': 56,
 'enter': 57,
 'at': 58,
 'was': 59,
 'oh': 60,
 'like': 61,
 'most': 62,
 'there': 63,
 'well': 64,
 'know': 65,
 'selfe': 66,
 'would': 67,
 'them': 68,
 'loue': 69,
 'may': 70,
 "'tis": 71,
 'vs': 72,
 'sir': 73,
 'qu': 74,
 'which': 75,
 'did': 76,
 'why': 77,
 'laer': 78,
 'giue': 79,
 'thee': 80,
 'ile': 81,
 'must': 82,
 'hath': 

In [4]:
# Step 1: Initialize a list to store input sequences
input_sequences = []  # This list will hold sequences of words for training

# Step 2: Loop through each line of the text
for line in text.split('\n'):  # Split text by newline to process line by line
    # Step 3: Convert the line into a sequence of word indices
    token_list = tokenizer.texts_to_sequences([line])[0]  # texts_to_sequences returns a list, take the first element

    # Step 4: Create n-gram sequences from the token list
    for i in range(1, len(token_list)):
        # Take tokens from start to i+1 to form an n-gram
        n_gram_sequence = token_list[:i+1]

        # Step 5: Append the sequence to the input_sequences list
        input_sequences.append(n_gram_sequence)

# Step 6: Check how many sequences were created
print(f'Total input sequences: {len(input_sequences)}')


Total input sequences: 25732



Input Sequences: Teaches the model the relationship between a context and the next word.

Sentence: "to be or not to be"
Tokens: [5, 12, 3, 7, 5, 12]
Generated Sequences:
[5, 12] -> Target: 3
[5, 12, 3] -> Target: 7
[5, 12, 3, 7] -> Target: 5
[5, 12, 3, 7, 5] -> Target: 12

In [5]:
# Step 1: Find the maximum sequence length
max_sequence_len = max([len(x) for x in input_sequences])
# This is needed because LSTM models expect all input sequences to have the same length
print(f"Maximum sequence length: {max_sequence_len}")

Maximum sequence length: 14


### **Why padding is  needed?**

**LSTM Requires Fixed-Length Input:**
- LSTM or RNN models require input sequences to be of the same length.
- However, the n-gram sequences we created have different lengths.

**Padding Solves the Problem:**
- We add zeros at the beginning of shorter sequences to make their length equal to `max_sequence_len`.

**Example:**
- Original sequence: `[5, 12, 3]`
- Max length = 6
- After padding: `[0, 0, 0, 5, 12, 3]`

**The Model Understands:**
- The model learns to ignore the zero padding and focuses on learning to predict the next word from the proper context.

In [6]:
# Step 1: Import pad_sequences (already imported earlier)
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Step 2: Pad all input sequences to have the same length
input_sequences = np.array(pad_sequences(input_sequences,
                                        maxlen=max_sequence_len,  # Pad all sequences to max length
                                        padding='pre'))           # Add padding at the beginning

# Step 3: Check the padded sequences
print(input_sequences[:5])  # Display first 5 sequences to see padding


[[   0    0    0    0    0    0    0    0    0    0    0    0    1  687]
 [   0    0    0    0    0    0    0    0    0    0    0    1  687    4]
 [   0    0    0    0    0    0    0    0    0    0    1  687    4   45]
 [   0    0    0    0    0    0    0    0    0    1  687    4   45   41]
 [   0    0    0    0    0    0    0    0    1  687    4   45   41 1886]]


In [7]:
# Step 1: Import TensorFlow (already imported)
import tensorflow as tf

# Step 2: Split input_sequences into predictors (X) and label (y)
x = input_sequences[:, :-1]  # All words except the last one are inputs (predictors)
y = input_sequences[:, -1]   # The last word in each sequence is the target (label)

# Step 3: Check shapes
print(f'X shape: {x.shape}')
print(f'y shape: {y.shape}')


X shape: (25732, 13)
y shape: (25732,)


### **Why is this step needed?**

**Predictors (X):**
- The model learns to predict the next word from these sequences.
- Example: `[0, 0, 0, 5, 12]` → The model will predict the next word based on this context.

**Label (y):**
- This is the target word that the model needs to predict.
- Example: From the sequence `[0, 0, 0, 5, 12, 3]`, the number `3` will be the target.

**Training-ready format:**
- Now we can one-hot encode `y` because the LSTM output will be categorical (with vocabulary size number of classes).
- This creates the final format needed for training the neural network.

In [8]:
# Step 1: One-hot encode the target labels
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
# Converts each integer label into a vector of length = total_words
# Example: if total_words = 100, label 5 → [0,0,0,0,0,1,0,...,0]

# Step 2: Check the shape of y
print(f'One-hot encoded y shape: {y.shape}')


One-hot encoded y shape: (25732, 4818)


# Split the data into training and testing sets

In [9]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

Model Trinig

In [10]:
# Step 1: Import EarlyStopping callback from Keras
from tensorflow.keras.callbacks import EarlyStopping

# Step 2: Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',        # Monitor the validation loss during training
    patience=3,                # Stop training if val_loss doesn't improve for 3 consecutive epochs
    restore_best_weights=True  # After stopping, restore model weights from the epoch with the best val_loss
)

# Step 3: Why we use it?
# - Prevents overfitting by stopping training early
# - Saves time by not training unnecessary epochs
# - Ensures the model keeps the best weights observed during training


In [11]:
# Step 1: Import necessary layers and model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Step 2: Define the Sequential model
model = Sequential()

# Step 3: Add Embedding layer
model.add(Embedding(
    input_dim=total_words,     # Vocabulary size
    output_dim=100,            # Embedding vector size
    input_length=max_sequence_len-1  # Input sequence length (excluding target word)
))
# Embedding layer converts word indices into dense vectors

# Step 4: Add first LSTM layer
model.add(LSTM(150, return_sequences=True))
# return_sequences=True because we will stack another LSTM on top

# Step 5: Add Dropout layer to prevent overfitting
model.add(Dropout(0.2))

# Step 6: Add second LSTM layer
model.add(LSTM(100))
# return_sequences=False by default, outputs the last hidden state

# Step 7: Add Dense output layer with softmax activation
model.add(Dense(total_words, activation="softmax"))
# Predicts probability for each word in the vocabulary

# Step 8: Compile the model
model.compile(
    loss="categorical_crossentropy",  # Suitable for multi-class classification
    optimizer='adam',                 # Adam optimizer for faster convergence
    metrics=['accuracy']              # Track accuracy during training
)

# Explicitly build the model with the input shape
model.build(input_shape=(None, max_sequence_len - 1))


# Step 9: Show model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# ## GRU RNN
# ## Define the model
# model=Sequential()
# model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
# model.add(GRU(150,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(GRU(100))
# model.add(Dense(total_words,activation="softmax"))

# # #Compile the model
# model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
# model.build(input_shape=(None, max_sequence_len-1))
# model.summary()

In [13]:
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 42s 59ms/step - accuracy: 0.0297 - loss: 7.1236 - val_accuracy: 0.0336 - val_loss: 6.7409
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 37s 57ms/step - accuracy: 0.0362 - loss: 6.4470 - val_accuracy: 0.0418 - val_loss: 6.8087
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 35s 54ms/step - accuracy: 0.0439 - loss: 6.3226 - val_accuracy: 0.0484 - val_loss: 6.8297
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 35s 54ms/step - accuracy: 0.0515 - loss: 6.1552 - val_accuracy: 0.0464 - val_loss: 6.8436
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 36s 56ms/step - accuracy: 0.0561 - loss: 6.0192 - val_accuracy: 0.0507 - val_loss: 6.8764
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - accuracy: 0.0616 - loss: 5.8769 - val_accuracy: 0.0554 - val_loss: 6.9746
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 36s 55ms/step - accuracy: 0.0700 - loss: 5.7608 - val_accuracy: 0.0618 - val_loss: 6.9991
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 36s 56ms/step - accuracy: 0.0747 - loss: 5.6113 - 

In [14]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [15]:
input_text="I love AI "
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:I love AI 
Next Word PRediction:haue


In [16]:
## Save the model
model.save("next_word_lstm.keras")
## Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
input_text="  Barn. Last night of all,When yond same"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next Word PRediction:{next_word}")

Input text:  Barn. Last night of all,When yond same
Next Word PRediction:scull


In [18]:
# Folder Create and download model

In [19]:
import os
print(os.listdir('.'))

['.config', 'next_word_lstm.keras', 'tokenizer.pickle', 'hamlet.txt', 'sample_data']


In [20]:
import zipfile

# List of files to include
files_to_zip = ['next_word_lstm.keras', 'tokenizer.pickle']
zip_filename = 'next_word_prediction_model.zip'

# Create a zip archive
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        zipf.write(file)  # Add file to zip

print(f"Created {zip_filename} containing the model and tokenizer.")


Created next_word_prediction_model.zip containing the model and tokenizer.


In [21]:
from google.colab import files
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the trained model
model = tf.keras.models.load_model('next_word_lstm.keras')

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

max_sequence_len = model.input_shape[1] + 1

# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_word_index:
            return word
    return None

# Streamlit app
st.title("Next Word Prediction using LSTM")

input_text = st.text_input("Enter a sentence:")

if input_text:
    next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
    if next_word:
        st.write(f"Next word prediction: **{next_word}**")
    else:
        st.write("Could not predict the next word.")

Writing app.py


In [23]:
!pip install streamlit
!streamlit run app.py &>/dev/null&

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.2 MB/s eta 0:00:00


In [24]:
import pkg_resources

# Define a list of the packages used in the notebook
packages = ['nltk', 'pandas', 'numpy', 'tensorflow', 'sklearn', 'streamlit', 'pickle']

# Print the version of each package if it's installed
for package in packages:
    try:
        print(f"{package}: {pkg_resources.get_distribution(package).version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Not installed")

/tmp/ipython-input-1104353115.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


nltk: 3.9.1
pandas: 2.2.2
numpy: 2.0.2
tensorflow: 2.19.0
sklearn: Not installed
streamlit: 1.50.0
pickle: Not installed


In [25]:
!pip install scikit-learn streamlit pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pickle5
  Running setup.py clean for pickle5
Failed to build pickle5
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pickle5)


In [26]:
import pkg_resources

# Define a list of the packages used in the notebook
packages = ['nltk', 'pandas', 'numpy', 'tensorflow', 'sklearn', 'streamlit', 'pickle']

# Print the version of each package if it's installed
for package in packages:
    try:
        print(f"{package}: {pkg_resources.get_distribution(package).version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package}: Not installed")

nltk: 3.9.1
pandas: 2.2.2
numpy: 2.0.2
tensorflow: 2.19.0
sklearn: Not installed
streamlit: 1.50.0
pickle: Not installed


In [27]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [28]:
!pip freeze > requirements.txt


In [29]:
# tokenizer JSON format save
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer_json)
